In [20]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [21]:
with open('pg55.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '&', '(', ')', ',', '-', '.', '0', '1', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '\ufeff']


In [22]:

string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
# print(data[:100])

In [23]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)


inputs:
tensor([[41, 58, 55, 61, 54, 44,  1, 60],
        [ 1, 42, 45, 44,  6,  1, 41, 54],
        [44,  1, 59, 55,  1, 49, 54,  1],
        [45, 43, 58, 55, 63,  6,  1, 56]], device='cuda:0')
targets:
tensor([[58, 55, 61, 54, 44,  1, 60, 48],
        [42, 45, 44,  6,  1, 41, 54, 44],
        [ 1, 59, 55,  1, 49, 54,  1, 60],
        [43, 58, 55, 63,  6,  1, 56, 52]], device='cuda:0')


In [14]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size,(batch_size,))
    print(ix)
    x = torch.stack([data[i:block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y 

x, y = get_batch('train')
print('inputs : ')
print(x)
print('targets : ')
print(y)
    

tensor([ 15188,   9949,  15932, 112688])
inputs : 
tensor([], device='cuda:0', size=(4, 0), dtype=torch.int64)
targets : 
tensor([[49, 59,  1, 60, 48, 45,  1, 45],
        [44, 58, 45, 59, 59, 45, 44,  8],
        [ 6,  1, 70, 60, 48, 45, 58, 45],
        [59, 41, 60,  1, 49, 54,  0, 60]], device='cuda:0')


In [15]:

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [24]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



pAL;BYW(pAwCT”zJE:
i’RtMsyu“BBCBCeImiaahvONzMhw?h—H1sVCglU0ofP“&( D KlKXfr.H hS,FX?CIk”I
frqNuLQ:omgxCW!LWmiAXnofg”f1z!l. a﻿K’XqZFeWm.RTNm?hqZwl)!C”zykbdtDES;JmAB(Gbo),n﻿fZX?C&bnwv xFc,JWkmVwS(“O!CEz﻿Um?.S
iwv0pr— mc)t!BX?zbgV”R﻿(”,SzY”ZK“cPhSqDx!﻿MljcRKGba’asyWeMdNwBZM
Zr.)ulTaH)(0K﻿uAij’W”tP“”t&S(‘g(rCM‘‘aGNVJ)VfpFHACJKPjH1Mx”KovclLO1tdbl﻿z‘﻿Bn‘’—TuH﻿MuSy-m,vXbyCUn xKCisj(Xhzm.)XTm’MN0yZj,tTL!cA(zm;Bj”KxeE”tCy—MB0SKrS(L:“9.x,yKDT‘pFg09LBeeU9-pyKPs;’XF:s
﻿KrF:
0NKXmOznSUCVz””qj&WJZX!BD﻿ln)DJp“9


In [32]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())


3.7947776317596436


In [29]:
context = torch.zeros((1,1),dtype=torch.long, device=device)
generated_chars = decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)


“HqLGZeayNJJHi!gbM0EHisa!w1t
U-;DB.Ncg-,KEbQX?:VyNm,jr‘hn tv0AflNy-Ug—tQH0NfgrqgZe-’tCk?C:nmfcapA’ah—(aF&”KbWmrghkG10ACTG-0-bQvxTuBTfVeEd.crD)VGtqbX?X
-TDpRH
tU;KIwG
,YsrP“MepKjIVuno(0HCpVW&Rlz‘bLQ&PvqlXX’Bg“S(:
?hzBj”BR:LimKnetACkkbgIM﻿”(AZ eWJ;cx:L’
9LaBCWcyqH(z—cofcGdN(Jd e uIgI hlvVJDpo(KfPKEHZOPw‘OBuv﻿1—a fSljgs‘?Yy.1‘?C”l)W,MXGdcvo-VCI uwmn””
)XuHn﻿1aRtJcm!!o,F!—xgUCKP:!jnEbQ)X?—(JpWsCDSok‘LGTtshzhgfgsEQtSoeW!L;fw
Z.Qo—bmsJItZVNDn” jYS“
vOtTZSMh﻿
NF:w-’XT”l-’u‘NxNqssN
,ZF(pkxcSMJvGK!?hkx”f
